In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from netCDF4 import Dataset

def parquet_to_netcdf(netcdf_file, headerdict):
    
    # Open a new NetCDF file for writing
    with Dataset(netcdf_file, 'w', format='NETCDF4') as ncfile:

        for key, value in headerdict.items():
            setattr(ncfile, key, value)

        mag_grp = ncfile.createGroup("magnetics")  

        pickup_group = mag_grp.createGroup("pickup")

        centrecol_group = pickup_group.createGroup("centrecolumn")
        cc_t1_group = centrecol_group.createGroup("t1")
        cc_t2_group = centrecol_group.createGroup("t2")

        outer_vessel_group = pickup_group.createGroup("outervessel")
        ov_t1_group = outer_vessel_group.createGroup("t1")
        ov_t2_group = outer_vessel_group.createGroup("t2")

        #############
        # Create compound types
        #############

        unitvector_dtype = np.dtype([("r", "<f8"),
                                     ("z", "<f8"),
                                     ("phi", "<f8")])

        coord_dtype = np.dtype([("r", "<f8"),
                                ("z", "<f8"),
                                ("phi", "<f8"),
                                ("theta", "<f8")])

        geom_dtype = np.dtype([("length", "<f8")])

        orientation_dtype = np.dtype([("measurement_direction", "S30"),
                                      ("unit_vector", unitvector_dtype)])

        bv_dtype = np.dtype([("name", "S50"),
                             ("version", "<f8"),
                             ("orientation", orientation_dtype),
                             ("coordinate", coord_dtype),
                             ("geometry", geom_dtype)])

        # Create the compound types in the file within the pickup group
        pickup_group.createCompoundType(unitvector_dtype, "UNIT_VECTOR")
        pickup_group.createCompoundType(orientation_dtype, "ORIENTATION")
        pickup_group.createCompoundType(coord_dtype, "COORDINATE")
        pickup_group.createCompoundType(geom_dtype, "GEOMETRY")
        lp_cp = pickup_group.createCompoundType(bv_dtype, "PICKUP_COIL")

        # Dimension
        pickup_group.createDimension('singleDim', 1)

        version = headerdict["version"] + 0.1 * headerdict["revision"]

        #############
        # Loop over rows in data, create variables in the appropriate group and fill data. 
        # This is repeated for each parquet file that needs to go into this one netcdf.
        #############

        # Read the ccbv Parquet file
        # ccbv is split into groups t1 and t2 for the two toroidal angles
        df = pd.read_parquet("geometry/data/amb/ccbv.parquet")
        for ind, row in df.iterrows():

            var = cc_t1_group.createVariable(row["uda_name"],
                                     lp_cp,
                                     ("singleDim",))

            data = np.empty(1, lp_cp.dtype_view)
            data["name"][:] = row["uda_name"]
            data["version"] = version
            data["coordinate"]["r"] = row["r"]
            data["coordinate"]["z"] = row["z"]
            data["coordinate"]["theta"] = row["poloidal_angle"]
            data["coordinate"]["phi"] = row["toroidal_angle1"]
            data["geometry"]["length"] = row["length"]

            data["orientation"]["measurement_direction"] = "PARALLEL"
            data["orientation"]["unit_vector"]["r"] = 0.
            data["orientation"]["unit_vector"]["phi"] = 0.
            data["orientation"]["unit_vector"]["z"] = 1.

            var[:] = data
            var.setncattr("units", "SI units: degrees, m")

        # Read the ccbv Parquet file
        # ccbv is split into groups t1 and t2 for the two toroidal angles
        df = pd.read_parquet("geometry/data/amb/ccbv.parquet")
        for ind, row in df.iterrows():

            var = cc_t2_group.createVariable(row["uda_name"],
                                     lp_cp,
                                     ("singleDim",))

            data = np.empty(1, lp_cp.dtype_view)
            data["name"][:] = row["uda_name"]
            data["version"] = version
            data["coordinate"]["r"] = row["r"]
            data["coordinate"]["z"] = row["z"]
            data["coordinate"]["theta"] = row["poloidal_angle"]
            data["coordinate"]["phi"] = row["toroidal_angle2"]
            data["geometry"]["length"] = row["length"]

            data["orientation"]["measurement_direction"] = "PARALLEL"
            data["orientation"]["unit_vector"]["r"] = 0.
            data["orientation"]["unit_vector"]["phi"] = 0.
            data["orientation"]["unit_vector"]["z"] = 1.

            var[:] = data
            var.setncattr("units", "SI units: degrees, m")

        # Read the obr Parquet file
        # obr is split into groups t1 and t2 for the two toroidal angles
        df = pd.read_parquet("geometry/data/amb/xma_obr.parquet")
        for ind, row in df.iterrows():

            var = ov_t1_group.createVariable(row["uda_name"],
                                     lp_cp,
                                     ("singleDim",))

            data = np.empty(1, lp_cp.dtype_view)
            data["name"][:] = row["uda_name"]
            data["version"] = version
            data["coordinate"]["r"] = row["r"]
            data["coordinate"]["z"] = row["z"]
            data["coordinate"]["theta"] = row["poloidal_angle"]
            data["coordinate"]["phi"] = row["toroidal_angle1"]
            data["geometry"]["length"] = row["length"]

            data["orientation"]["measurement_direction"] = "PARALLEL"
            data["orientation"]["unit_vector"]["r"] = 0.
            data["orientation"]["unit_vector"]["phi"] = 0.
            data["orientation"]["unit_vector"]["z"] = 1.

            var[:] = data
            var.setncattr("units", "SI units: degrees, m")

        # Read the obr Parquet file
        # obr is split into groups t1 and t2 for the two toroidal angles
        df = pd.read_parquet("geometry/data/amb/xma_obr.parquet")
        for ind, row in df.iterrows():

            var = ov_t2_group.createVariable(row["uda_name"],
                                     lp_cp,
                                     ("singleDim",))

            data = np.empty(1, lp_cp.dtype_view)
            data["name"][:] = row["uda_name"]
            data["version"] = version
            data["coordinate"]["r"] = row["r"]
            data["coordinate"]["z"] = row["z"]
            data["coordinate"]["theta"] = row["poloidal_angle"]
            data["coordinate"]["phi"] = row["toroidal_angle2"]
            data["geometry"]["length"] = row["length"]

            data["orientation"]["measurement_direction"] = "PARALLEL"
            data["orientation"]["unit_vector"]["r"] = 0.
            data["orientation"]["unit_vector"]["phi"] = 0.
            data["orientation"]["unit_vector"]["z"] = 1.

            var[:] = data
            var.setncattr("units", "SI units: degrees, m")

        # Read the obv Parquet file
        # obv is split into groups t1 and t2 for the two toroidal angles
        df = pd.read_parquet("geometry/data/amb/xma_obv.parquet")
        for ind, row in df.iterrows():

            var = ov_t1_group.createVariable(row["uda_name"],
                                     lp_cp,
                                     ("singleDim",))

            data = np.empty(1, lp_cp.dtype_view)
            data["name"][:] = row["uda_name"]
            data["version"] = version
            data["coordinate"]["r"] = row["r"]
            data["coordinate"]["z"] = row["z"]
            data["coordinate"]["theta"] = row["poloidal_angle"]
            data["coordinate"]["phi"] = row["toroidal_angle1"]
            data["geometry"]["length"] = row["length"]

            data["orientation"]["measurement_direction"] = "PARALLEL"
            data["orientation"]["unit_vector"]["r"] = 0.
            data["orientation"]["unit_vector"]["phi"] = 0.
            data["orientation"]["unit_vector"]["z"] = 1.

            var[:] = data
            var.setncattr("units", "SI units: degrees, m")

        # Read the obv Parquet file
        # obv is split into groups t1 and t2 for the two toroidal angles
        df = pd.read_parquet("geometry/data/amb/xma_obv.parquet")
        for ind, row in df.iterrows():

            var = ov_t2_group.createVariable(row["uda_name"],
                                     lp_cp,
                                     ("singleDim",))

            data = np.empty(1, lp_cp.dtype_view)
            data["name"][:] = row["uda_name"]
            data["version"] = version
            data["coordinate"]["r"] = row["r"]
            data["coordinate"]["z"] = row["z"]
            data["coordinate"]["theta"] = row["poloidal_angle"]
            data["coordinate"]["phi"] = row["toroidal_angle2"]
            data["geometry"]["length"] = row["length"]

            data["orientation"]["measurement_direction"] = "PARALLEL"
            data["orientation"]["unit_vector"]["r"] = 0.
            data["orientation"]["unit_vector"]["phi"] = 0.
            data["orientation"]["unit_vector"]["z"] = 1.

            var[:] = data
            var.setncattr("units", "SI units: degrees, m")

if __name__ == "__main__":

    # Metadata for the netcdf file
    headerdict = {
        "Conventions": "",
        "device": "MAST",
        "class": "magnetics",
        "system": "pickup",
        "configuration": "geometry",
        "shotRangeStart": 0,
        "shotRangeStop": 400000,
        "content": "geometry of the magnetic pickup coils for MAST",
        "comment": "",
        "units": "SI, degrees, m",
        "coordinateSystem": "Cylindrical",
        "structureCastType": "unknown",
        "calibration": "None",
        "version": 0,
        "revision": 0,
        "status": "development",
        "releaseDate": datetime.strftime(datetime.now(), "%Y-%m-%d"),
        "releaseTime": datetime.strftime(datetime.now(), "%H:%M:%S"),
        "owner": "jhodson",
        "signedOffBy": "",
        "signedOffDate": "",
        "creatorCode": "python create_netcdf_pickup.py",
        "creationDate": datetime.strftime(datetime.now(), "%Y-%m-%d"),
        "createdBy": "jhodson",
        "testCode": "",
        "testDate": "",
        "testedBy": ""
    }

    # Example usage
    parquet_to_netcdf("pickup_coils.nc", headerdict)
